In [ ]:
import pandas as pd
from functions.processing_data_functions import standardize_death_types, calculate_age_specific_rates

# Load the Excel file
file_path = "Data/deaths_in_custody_by_prison.xlsx" # extracte from the Deaths data tool

# Read the Excel file into a Pandas DataFrame
data = pd.read_excel(file_path)

# Standardize column names for ease of use
data.columns = [col.strip().lower().replace(" ", "_") for col in data.columns]

In [ ]:
# Apply the standardization
data = standardize_death_types(data)

# Verify the unique values
print(data.type_of_death.unique())

In [ ]:
# Generate separate dataframes for specific causes
# Collapse deaths grouped by prison and date, removing age_band from grouping
all_causes = (
    data.groupby(["prison", "date"])
    .agg({"incidents": "sum"})
    .reset_index()
)

natural_causes = data[
    data["type_of_death"].str.contains("natural", case=False, na=False)
].groupby(["prison", "date"]).agg({"incidents": "sum"}).reset_index()

self_inflicted = data[
    data["type_of_death"].str.contains("self-inflicted", case=False, na=False)
].groupby(["prison", "date"]).agg({"incidents": "sum"}).reset_index()

homicides = data[
    data["type_of_death"].str.contains("homicide", case=False, na=False)
].groupby(["prison", "date"]).agg({"incidents": "sum"}).reset_index()

other_causes = data[
    ~data["type_of_death"].str.contains("natural|self-inflicted|homicide", case=False, na=False)
].groupby(["prison", "date"]).agg({"incidents": "sum"}).reset_index()

# Display the structure and a few rows of each dataframe
print("Collapsed deaths grouped by prison and date:")
print(all_causes.head())

print("Natural causes grouped by prison and date:")
print(natural_causes.head())

print("Self-inflicted deaths grouped by prison and date:")
print(self_inflicted.head())

print("Homicides grouped by prison and date:")
print(homicides.head())

print("Other causes grouped by prison and date:")
print(other_causes.head())

# Save the dataframes for future use
all_causes.to_csv("Output/Deaths/collapsed_deaths_by_prison_date.csv", index=False)
natural_causes.to_csv("Output/Deaths/natural_causes_by_prison_date.csv", index=False)
self_inflicted.to_csv("Output/Deaths/self_inflicted_by_prison_date.csv", index=False)
homicides.to_csv("Output/Deaths/homicides_by_prison_date.csv", index=False)
other_causes.to_csv("Output/Deaths/other_causes_by_prison_date.csv", index=False)


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import stats

# Set style for better visualizations with colorblind-friendly palette
sns.set_style('whitegrid')
plt.rcParams.update({'font.size': 12})
# Use a colorblind-friendly palette
# Using the 'colorblind' palette from seaborn
colors = sns.color_palette("colorblind")

# Read the data
file_path = 'Output/Deaths/collapsed_deaths_by_prison_date.csv'
df = pd.read_csv(file_path)

# Convert the date column to datetime format
df['date'] = pd.to_datetime(df['date'])

# Group by month and sum incidents
# First create a monthly date series
monthly_deaths = df.groupby(pd.Grouper(key='date', freq='M'))['incidents'].sum().reset_index()

# Create a time series plot
plt.figure(figsize=(15, 8))
plt.plot(monthly_deaths['date'], monthly_deaths['incidents'], marker='o', linestyle='-', color=colors[0], linewidth=2)

# Add a trend line using rolling mean with 95% CI
window_size = 12
rolling_mean = monthly_deaths['incidents'].rolling(window=window_size).mean()

# Calculate the standard error for the rolling mean
rolling_std = monthly_deaths['incidents'].rolling(window=window_size).std()
rolling_se = rolling_std / np.sqrt(window_size)

# Calculate 95% confidence interval
conf_interval = 1.96 * rolling_se  # 1.96 is the z-score for 95% confidence

# Plot rolling mean and confidence interval
plt.plot(monthly_deaths['date'], rolling_mean, color=colors[1], linewidth=2, label='12-month rolling average')
plt.fill_between(monthly_deaths['date'], 
                 rolling_mean - conf_interval, 
                 rolling_mean + conf_interval, 
                 color=colors[1], alpha=0.2)

# Calculate yearly totals for statistics only
yearly_deaths = df.groupby(df['date'].dt.year)['incidents'].sum()

# Add title and labels
plt.title('Total Prison Deaths by Month', fontsize=16, fontweight='bold')
plt.xlabel('Date', fontsize=14)
plt.ylabel('Number of Deaths', fontsize=14)
plt.grid(True, alpha=0.3)
plt.legend()

# Rotate x-axis labels for better readability and format them as numeric
plt.xticks(rotation=45)

# Format the x-axis date labels to use numeric month format (MM/YYYY)
import matplotlib.dates as mdates
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%m/%Y'))

# Adjust layout
plt.tight_layout()

# Add annotation only for the highest peak
highest_peak = monthly_deaths.nlargest(1, 'incidents').iloc[0]
# Format date as numeric month and year (e.g., "01/2005" for January 2005)
month_year = highest_peak['date'].strftime('%m/%Y')
plt.annotate(f"Peak: {highest_peak['incidents']} ({month_year})",
            xy=(highest_peak['date'], highest_peak['incidents']),
            xytext=(10, 15),
            textcoords='offset points',
            arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=.2', color=colors[3]))

# Save the figure
plt.savefig('Output/Deaths/prison_deaths_over_time.png', dpi=300, bbox_inches='tight')

# Show the plot
plt.show()

# Additional analysis: Calculate quarterly and yearly statistics
quarterly_deaths = df.groupby(pd.Grouper(key='date', freq='Q'))['incidents'].sum()
print("Quarterly statistics:")
print(quarterly_deaths.describe())

print("\nYearly statistics:")
print(yearly_deaths.describe())

# Create a bar chart of yearly deaths
plt.figure(figsize=(15, 6))
yearly_deaths.plot(kind='bar', color=colors[0])
plt.title('Total Prison Deaths by Year', fontsize=16, fontweight='bold')
plt.xlabel('Year', fontsize=14)
plt.ylabel('Number of Deaths', fontsize=14)
plt.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.savefig('Output/Deaths/prison_deaths_yearly.png', dpi=300, bbox_inches='tight')
plt.show()

### Calculating the population by age for years 2002-2024
Source https://commonslibrary.parliament.uk/research-briefings/sn04334/

In [ ]:
import pandas as pd

# Create the DataFrame directly from the data
df = {
    'Year': ['2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', 
             '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021',
             '2022', '2023', '2024'],
    'Age_15_20': [10923, 10199, 10011, 10138, 10443, 11045, 11352, 10892, 10234, 9508,
                  8725, 7138, 6442, 5730, 5194, 5219, 4883, 4770, 4068, 3883,
                  3573, 3625, 3354],
    'Age_21_29': [26710, 27584, 27359, 27273, 27512, 27843, 29222, 29386, 29960, 29538,
                  29580, 28217, 28609, 28027, 26758, 26020, 24333, 23918, 22371, 21530,
                  21301, 21590, 21381],
    'Age_30_39': [20391, 21654, 21877, 22007, 21960, 21413, 22073, 21699, 22220, 22740,
                  22900, 23248, 24361, 25075, 25189, 25894, 25310, 25745, 25307, 25211, 
                  26446, 28524, 29339],
    'Age_40_49': [8371, 9210, 9973, 11068, 11881, 12638, 13400, 13866, 14325, 14645,
                  15116, 15008, 15017, 15381, 15283, 15354, 14631, 14660, 14497, 14569,
                  15504, 17174, 18175],
    'Age_50+': [4824, 5010, 5268, 5704, 6186, 6795, 7147, 7548, 8263, 8943,
               9727, 10231, 11080, 11980, 12710, 13376, 13616, 13617, 13271, 13131,
               13835, 14938, 15620],
    'Total': [71218, 73657, 74488, 76190, 77982, 79734, 83194, 83391, 85002, 85374,
             86048, 83842, 85509, 86193, 85134, 85863, 82773, 82710, 79514, 78324,
             80659, 85851, 87869]
}

# Create the DataFrame
prison_df = pd.DataFrame(df)

# Add columns for proportions of each age group
age_columns = ['Age_15_20', 'Age_21_29', 'Age_30_39', 'Age_40_49', 'Age_50+']

# Calculate proportions and add to DataFrame
for age_col in age_columns:
    prop_col_name = f'Prop_{age_col}'
    prison_df[prop_col_name] = prison_df[age_col] / prison_df['Total']
    # Format as percentage (optional)
    prison_df[prop_col_name] = prison_df[prop_col_name].round(4)  # Round to 4 decimal places (0.1234 = 12.34%)

# Display the first few rows with the new proportion columns
print(prison_df.head())

# Save to CSV if needed
prison_df.to_csv('Data/prison_population_with_proportions.csv', index=False)

In [ ]:
prison_df

### Extracting data for causes of death per age

In [ ]:
import pandas as pd

# Group the data by year and type_of_death
deaths_by_cause = data.groupby(['year', 'type_of_death']).agg(
    total_deaths=('incidents', 'sum')
).reset_index()

# Pivot the data to have years as rows and causes of death as columns
deaths_yearly = deaths_by_cause.pivot_table(
    index='year',
    columns='type_of_death',
    values='total_deaths',
    fill_value=0
).reset_index()

# Make sure column names are properly formatted
deaths_yearly.columns.name = None

# Add a total column
cause_columns = [col for col in deaths_yearly.columns if col != 'year']
deaths_yearly['Total'] = deaths_yearly[cause_columns].sum(axis=1)

# Sort by year
deaths_yearly = deaths_yearly.sort_values('year')

# Print the first few rows
print("Deaths by Year and Cause:")
print(deaths_yearly.head())

# Save to CSV
deaths_yearly.to_csv('Output/Deaths/prison_deaths_yearly.csv', index=False)

In [ ]:
# Convert 'Year' column in prison_df to numeric for proper merging
# Handle the '2024 (March)' special case
prison_df['Year_numeric'] = prison_df['Year'].apply(lambda x: int(x.split()[0]))

# Assuming deaths_yearly is already created from your previous code
# If not, you would have already processed it using your code

# Now perform the left join
# Note: We're assuming the 'year' column in deaths_yearly is already numeric
merged_df = pd.merge(
    prison_df,
    deaths_yearly,
    left_on='Year_numeric',
    right_on='year',
    how='left'
)

In [ ]:
print(merged_df.head())

### Additionally grouping by age band

In [ ]:
import pandas as pd

# Group the data by year, age_band, and type_of_death
deaths_by_age_cause = data.groupby(['year', 'age_band', 'type_of_death']).agg(
    total_deaths=('incidents', 'sum')
).reset_index()

# Pivot the data to have year and age_band as rows and causes of death as columns
deaths_age_yearly = deaths_by_age_cause.pivot_table(
    index=['year', 'age_band'],
    columns='type_of_death',
    values='total_deaths',
    fill_value=0
).reset_index()

# Make sure column names are properly formatted
deaths_age_yearly.columns.name = None

# Add a total column
cause_columns = [col for col in deaths_age_yearly.columns if col not in ['year', 'age_band']]
deaths_age_yearly['Total'] = deaths_age_yearly[cause_columns].sum(axis=1)

# Sort by year and age_band
deaths_age_yearly = deaths_age_yearly.sort_values(['year', 'age_band'])

# Print the first few rows
print("Deaths by Year, Age Band, and Cause:")
print(deaths_age_yearly.head())

# Save to CSV
deaths_age_yearly.to_csv('Output/Deaths/prison_deaths_age_yearly.csv', index=False)

# Optional: Create a version with just the totals by age band for each year
deaths_by_age_year = data.groupby(['year', 'age_band']).agg(
    total_deaths=('incidents', 'sum')
).reset_index()

deaths_by_age_year_pivot = deaths_by_age_year.pivot_table(
    index='year',
    columns='age_band',
    values='total_deaths',
    fill_value=0
).reset_index()

deaths_by_age_year_pivot.columns.name = None

# Add a total column for all age bands
age_columns = [col for col in deaths_by_age_year_pivot.columns if col != 'year']
deaths_by_age_year_pivot['Total'] = deaths_by_age_year_pivot[age_columns].sum(axis=1)

# Sort by year
deaths_by_age_year_pivot = deaths_by_age_year_pivot.sort_values('year')

print("\nTotal Deaths by Year and Age Band:")
print(deaths_by_age_year_pivot.head())

# Save to CSV
deaths_by_age_year_pivot.to_csv('Output/Deaths/prison_deaths_by_age_year.csv', index=False)

### Analyse by cause of deaths and age

In [ ]:
print(data.head())
print(prison_df.head())

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# First, we need to map the age bands in 'data' to match the age categories in 'prison_df'
# Create a mapping dictionary
age_mapping = {
    'under 21': 'Age_15_20',
    '18 to 20': 'Age_15_20',  # If there are more specific categories in your data
    '21 to 24': 'Age_21_29',
    '25 to 29': 'Age_21_29',
    '30 to 39': 'Age_30_39',
    '40 to 49': 'Age_40_49',
    '50 to 59': 'Age_50+',
    '60 to 69': 'Age_50+',
    '70+': 'Age_50+',
    '70 and over': 'Age_50+'
}

# Add a standardized age category to the deaths data
data['age_category'] = data['age_band'].map(age_mapping)

# Group deaths by year, standardized age category, and type of death
deaths_by_age = data.groupby(['year', 'age_category', 'type_of_death']).agg(
    total_deaths=('incidents', 'sum')
).reset_index()

# Filter out any rows with NaN in age_category (in case some age bands didn't match)
deaths_by_age = deaths_by_age.dropna(subset=['age_category'])


# Calculate age-specific death rates
age_specific_rates = calculate_age_specific_rates(deaths_by_age, prison_df)

# Display some results
print("Age-specific death rates per 1000 prisoners:")
print(age_specific_rates.head(20))

# Create visualizations

# 1. Death rates by age group and cause over time
plt.figure(figsize=(14, 10))

for i, cause in enumerate(['Natural Causes', 'Self-Inflicted', 'Other', 'Homicide']):
    plt.subplot(2, 2, i+1)
    
    # Filter data for this cause
    cause_data = age_specific_rates[age_specific_rates['cause_of_death'] == cause]
    
    # Pivot for plotting
    pivot_data = cause_data.pivot(index='year', columns='age_category', values='rate_per_1000')
    
    # Plot
    pivot_data.plot(marker='o', ax=plt.gca())
    plt.title(f'{cause} Death Rate by Age Group')
    plt.ylabel('Deaths per 1000 prisoners')
    plt.xlabel('Year')
    plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('Data/death_rates_by_age_cause.png', dpi=300)

# 2. Overall trends in death rates by cause
plt.figure(figsize=(10, 6))

# Calculate overall rates by cause and year
overall_rates = age_specific_rates.groupby(['year', 'cause_of_death']).apply(
    lambda x: (x['deaths'].sum() / x['population'].sum()) * 1000
).reset_index(name='overall_rate')

# Pivot for plotting
overall_pivot = overall_rates.pivot(index='year', columns='cause_of_death', values='overall_rate')

# Plot
overall_pivot.plot(marker='o')
plt.title('Overall Death Rates by Cause')
plt.ylabel('Deaths per 1000 prisoners')
plt.xlabel('Year')
plt.grid(True, alpha=0.3)
plt.legend(title='Cause of Death')

plt.tight_layout()
plt.savefig('Data/overall_death_rates.png', dpi=300)

# 3. Heatmap of death rates by age and cause (averaged across years)
plt.figure(figsize=(10, 8))

# Calculate average rates across years
avg_rates = age_specific_rates.groupby(['age_category', 'cause_of_death'])['rate_per_1000'].mean().reset_index()

# Pivot for heatmap
heatmap_data = avg_rates.pivot(index='age_category', columns='cause_of_death', values='rate_per_1000')

# Plot heatmap
sns.heatmap(heatmap_data, annot=True, fmt='.2f', cmap='YlOrRd')
plt.title('Average Death Rates by Age Group and Cause')
plt.ylabel('Age Group')
plt.xlabel('Cause of Death')

plt.tight_layout()
plt.savefig('Data/death_rates_heatmap.png', dpi=300)

# Save the detailed results to CSV
age_specific_rates.to_csv('Data/age_specific_death_rates.csv', index=False)

# Calculate age-standardized death rates to control for changing age structure
def calculate_standardized_rates(rates_df, population_df):
    # For standard population, use the first year in the dataset
    first_year = population_df['Year'].astype(str).min()
    if 'Year_numeric' in population_df.columns:
        standard_year = population_df[population_df['Year'] == first_year]['Year_numeric'].iloc[0]
    else:
        standard_year = int(first_year)
    
    # Get age distribution for standard year
    standard_pop = population_df[population_df['Year'] == first_year].iloc[0]
    
    results = []
    
    # For each year and cause of death
    for year in rates_df['year'].unique():
        for cause in rates_df['cause_of_death'].unique():
            # Get rates for this year and cause
            year_cause_rates = rates_df[(rates_df['year'] == year) & 
                                       (rates_df['cause_of_death'] == cause)]
            
            # Calculate expected deaths using standard population
            expected_deaths = 0
            total_standard_pop = 0
            
            for age_cat in ['Age_15_20', 'Age_21_29', 'Age_30_39', 'Age_40_49', 'Age_50+']:
                # Get rate for this age category
                age_rate = year_cause_rates[year_cause_rates['age_category'] == age_cat]['rate_per_1000'].iloc[0] if not year_cause_rates[year_cause_rates['age_category'] == age_cat].empty else 0
                
                # Get standard population for this age category
                std_pop = standard_pop[age_cat]
                total_standard_pop += std_pop
                
                # Calculate expected deaths
                expected_deaths += (age_rate / 1000) * std_pop
            
            # Calculate standardized rate
            std_rate = (expected_deaths / total_standard_pop) * 1000
            
            results.append({
                'year': year,
                'cause_of_death': cause,
                'standardized_rate': std_rate
            })
    
    return pd.DataFrame(results)

# Calculate age-standardized rates
try:
    standardized_rates = calculate_standardized_rates(age_specific_rates, prison_df)
    
    # Plot standardized vs. crude rates
    plt.figure(figsize=(14, 10))
    
    for i, cause in enumerate(['Natural Causes', 'Self-Inflicted', 'Other', 'Homicide']):
        plt.subplot(2, 2, i+1)
        
        # Get standardized rates for this cause
        std_data = standardized_rates[standardized_rates['cause_of_death'] == cause]
        
        # Get crude rates for this cause
        crude_data = overall_rates[overall_rates['cause_of_death'] == cause]
        
        # Plot both
        plt.plot(std_data['year'], std_data['standardized_rate'], marker='o', label='Age-standardised')
        plt.plot(crude_data['year'], crude_data['overall_rate'], marker='s', label='Crude rate')
        
        plt.title(f'{cause} Death Rates')
        plt.ylabel('Deaths per 1000 prisoners')
        plt.xlabel('Year')
        plt.grid(True, alpha=0.3)
        plt.legend()
    
    plt.tight_layout()
    plt.savefig('Output/Deaths/standardized_vs_crude_rates.png', dpi=300)
    
    # Save standardized rates
    standardized_rates.to_csv('Output/Deaths/standardized_death_rates.csv', index=False)
    
except Exception as e:
    print(f"Could not calculate standardized rates: {e}")

print("Analysis complete. Results saved to Ouput/Deaths/ directory.")

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# -------- MOVING AVERAGE CALCULATION --------

# 3-year moving average for age-specific rates
smoothed_rates = age_specific_rates.copy()
smoothed_rates['rate_per_1000_ma'] = smoothed_rates.sort_values('year').groupby(
    ['age_category', 'cause_of_death']
)['rate_per_1000'].transform(lambda x: x.rolling(window=3, min_periods=1).mean())

# 3-year moving average for overall crude rates
smoothed_overall = overall_rates.copy()
smoothed_overall['overall_rate_ma'] = smoothed_overall.sort_values('year').groupby(
    'cause_of_death'
)['overall_rate'].transform(lambda x: x.rolling(window=3, min_periods=1).mean())

# 3-year moving average for standardized rates
smoothed_std = standardized_rates.copy()
smoothed_std['standardized_rate_ma'] = smoothed_std.sort_values('year').groupby(
    'cause_of_death'
)['standardized_rate'].transform(lambda x: x.rolling(window=3, min_periods=1).mean())

# -------- CLIP AND SET AXIS RANGE --------

x_min, x_max = 2002, 2025
x_ticks = np.arange(x_min, x_max + 1, 2)

smoothed_rates_clipped = smoothed_rates[(smoothed_rates['year'] >= x_min) & (smoothed_rates['year'] <= 2024)]
smoothed_overall_clipped = smoothed_overall[(smoothed_overall['year'] >= x_min) & (smoothed_overall['year'] <= 2024)]
smoothed_std_clipped = smoothed_std[(smoothed_std['year'] >= x_min) & (smoothed_std['year'] <= 2024)]

# -------- 1. DEATH RATES BY AGE AND CAUSE --------

plt.figure(figsize=(14, 10))

for i, cause in enumerate(['Natural Causes', 'Self-Inflicted', 'Other', 'Homicide']):
    ax = plt.subplot(2, 2, i+1)
    cause_data = smoothed_rates_clipped[smoothed_rates_clipped['cause_of_death'] == cause]
    pivot_data = cause_data.pivot(index='year', columns='age_category', values='rate_per_1000_ma')
    pivot_data.plot(marker='o', ax=ax)
    ax.set_title(f'{cause} Death Rate by Age Group (3Y MA)')
    ax.set_ylabel('Deaths per 1000 prisoners')
    ax.set_xlabel('Year')
    ax.grid(True, alpha=0.3)
    ax.set_xlim(x_min, x_max)
    ax.set_xticks(x_ticks)

plt.tight_layout()
plt.savefig('Output/Deaths/death_rates_by_age_cause_ma_clipped.png', dpi=300)

# -------- 2. OVERALL DEATH RATES BY CAUSE --------

plt.figure(figsize=(10, 6))
ax = smoothed_overall_clipped.pivot(index='year', columns='cause_of_death', values='overall_rate_ma').plot(marker='o')
ax.set_title('Overall Death Rates by Cause (3Y MA)')
ax.set_ylabel('Deaths per 1000 prisoners')
ax.set_xlabel('Year')
ax.grid(True, alpha=0.3)
ax.set_xlim(x_min, x_max)
ax.set_xticks(x_ticks)
plt.legend(title='Cause of Death')
plt.tight_layout()
plt.savefig('Output/Deaths/overall_death_rates_ma_clipped.png', dpi=300)

# -------- 3. HEATMAP (average of smoothed rates across years) --------

plt.figure(figsize=(10, 8))
avg_rates_ma = smoothed_rates_clipped.groupby(['age_category', 'cause_of_death'])['rate_per_1000_ma'].mean().reset_index()
heatmap_data_ma = avg_rates_ma.pivot(index='age_category', columns='cause_of_death', values='rate_per_1000_ma')

sns.heatmap(heatmap_data_ma, annot=True, fmt='.2f', cmap='YlOrRd')
plt.title('Avg Death Rates by Age Group and Cause (3-Year MA)')
plt.ylabel('Age Group')
plt.xlabel('Cause of Death')
plt.tight_layout()
plt.savefig('Output/Deaths/death_rates_heatmap_ma.png', dpi=300)

# -------- 4. STANDARDIZED VS CRUDE RATES --------

plt.figure(figsize=(14, 10))
for i, cause in enumerate(['Natural Causes', 'Self-Inflicted', 'Other', 'Homicide']):
    ax = plt.subplot(2, 2, i+1)
    std_data = smoothed_std_clipped[smoothed_std_clipped['cause_of_death'] == cause]
    crude_data = smoothed_overall_clipped[smoothed_overall_clipped['cause_of_death'] == cause]
    ax.plot(std_data['year'], std_data['standardized_rate_ma'], marker='o', label='Age-standardised (3Y MA)')
    ax.plot(crude_data['year'], crude_data['overall_rate_ma'], marker='s', label='Crude rate (3Y MA)')
    ax.set_title(f'{cause}')
    ax.set_ylabel('Deaths per 1000 prisoners')
    ax.set_xlabel('Year')
    ax.grid(True, alpha=0.3)
    ax.set_xlim(x_min, x_max)
    ax.set_xticks(x_ticks)
    ax.legend()

plt.tight_layout()
plt.savefig('Output/Deaths/standardized_vs_crude_rates_ma_clipped.png', dpi=300)
